In [36]:
import tkinter as tk
from tkinter import filedialog, simpledialog
import cv2 # type: ignore
import pytesseract # type: ignore
import re
import datetime
import vobject # type: ignore
import csv

In [37]:
# Set Tesseract path on Windows (not required on Linux)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [38]:
# Extract name from the text
def extract_name_from_text(text):
    # Use OCR to extract the first name from the text
    lines = text.split("\n")
    for line in lines:
        if line.strip():  # If the line contains text
            name = line.strip()
            return name
    return None

In [39]:
# Process the image and extract text
def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale to improve text recognition
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use OCR to extract text from the image
    extracted_text = pytesseract.image_to_string(gray)
    
    # Extract phone numbers using regular expressions (RegEx)
   # numbers = re.findall(r"\+?\d{10,15}", extracted_text)  # Looking for phone numbers
    numbers = re.findall(r"(?:\+?20|420)?(01[0125]\d{8})\b", extracted_text)
    
    # Ensure there are numbers to extract
    if not numbers:
        print("❌ No phone numbers were found in the image!")
        return
    
    # Try to extract names from the text (if any)
    name = extract_name_from_text(extracted_text)
    
    if not name:
        # If no name is found, ask the user to input a default name
        name = simpledialog.askstring("Enter Name", "No name found in the image. Please enter a default name:", parent=root)
        if not name:
            name = "Default"  # If no name is entered, use the default name
    
    # Generate a filename based on the current date and time
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create the VCF file
    vcf_filename = f"contacts_{timestamp}.vcf"
    with open(vcf_filename, "w", encoding="utf-8") as vcf_file:
        for i, number in enumerate(numbers, start=1):
            contact = vobject.vCard()
            contact.add('fn').value = f"{name} {i}"  # Assign the name
            contact.add('tel').value = number  # Add the phone number
            vcf_file.write(contact.serialize())  # Save the contact in the file
    
    # Create the CSV file
    csv_filename = f"contacts_{timestamp}.csv"
    with open(csv_filename, "w", newline='', encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Name", "Phone Number", "Note"])  # Column headers
        for i, number in enumerate(numbers, start=1):
            writer.writerow([f"{name} {i}", number])  # Write phone numbers
    
    print(f"✅ Extracted {len(numbers)} phone numbers and created the files successfully!")
    print(f"📥 You can now download the files: {vcf_filename}, {csv_filename}")

In [40]:
# Upload image using file dialog
def upload_image():
    file_path = filedialog.askopenfilename(title="Choose an Image", filetypes=(("Images", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")))
    if file_path:
        process_image(file_path) 

In [41]:
# Animate button color and size for hover effect
def on_enter_button(e):
    e.widget['background'] = '#45a049'
    e.widget['width'] = 18

def on_leave_button(e):
    e.widget['background'] = '#4CAF50'
    e.widget['width'] = 15

# Create the GUI
def create_gui():
    global root
    root = tk.Tk()
    root.title("Image to Contacts Extractor")
    
    # Set dark mode theme
    root.tk_setPalette(background="#333333")
    
    # Center the window
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    window_width = 400
    window_height = 250
    position_top = int(screen_height / 2 - window_height / 2)
    position_right = int(screen_width / 2 - window_width / 2)
    root.geometry(f'{window_width}x{window_height}+{position_right}+{position_top}')
    
    # Modern background and rounded corners
    root.configure(bg="#333333")
    
    # Upload image button
    upload_button = tk.Button(root, text="Upload Image", command=upload_image, bg="#4CAF50", fg="white", font=("Arial", 14), relief="flat", bd=0)
    upload_button.pack(pady=20)
    upload_button.bind("<Enter>", on_enter_button)
    upload_button.bind("<Leave>", on_leave_button)
    
    # Exit button
    exit_button = tk.Button(root, text="Exit", command=root.quit, bg="#f44336", fg="white", font=("Arial", 14), relief="flat", bd=0)
    exit_button.pack(pady=20)
    exit_button.bind("<Enter>", on_enter_button)
    exit_button.bind("<Leave>", on_leave_button)
    
    # Add an animated label (you can change the text)
    label = tk.Label(root, text="Welcome to the Image to Contacts Extractor", fg="white", font=("Arial", 12), bg="#333333")
    label.pack(pady=10)
    
    animate_label(label, 0)  # Start the animation on label
    
    root.mainloop()

# Animation for the label text
def animate_label(label, index):
    text = "Welcome to the Image to Contacts Extractor"
    label.config(text=text[:index])
    if index < len(text):
        root.after(150, animate_label, label, index + 1)

# Run the GUI
create_gui()


✅ Extracted 109 phone numbers and created the files successfully!
📥 You can now download the files: contacts_20250226_151901.vcf, contacts_20250226_151901.csv


In [30]:
import re

# Example extracted text with different number formats
extracted_text = """
    VODAFONE EG 1,201000431458,Created automatically by Python
    VODAFONE EG 2,201000741823,Created automatically by Python
    VODAFONE EG 3,201001642453,Created automatically by Python
    VODAFONE EG 4,4201003573538,Created automatically by Python
    VODAFONE EG 5,+201125678912,Created automatically by Python
"""

# Regular expression to extract numbers starting with 012, 011, 015, or 010
numbers = re.findall(r"(?:\+?20|420)?(01[0125]\d{8})\b", extracted_text)

print(numbers)  # Expected Output: ['01000431458', '01000741823', '01001642453', '01003573538', '01125678912']


['01000431458', '01000741823', '01001642453', '01003573538', '01125678912']
